# Robust location-transportation problem

**Author**: Sebastian Granda

**Course**: Stochastic Optimization

**Reference**: Zeng, B., & Zhao, L. (2013). Solving two-stage robust optimization problems using a column-and-constraint generation method. Operations Research Letters, 41(5), 457-461.

## Implementation notes

This project has been implemented as a Julia package called `LocationTransportation`. The entry-point is the `execute` function, which creates an `Input` instance, solves the problem, and returns the solution as a `Solution` object. 

To use the package, it is required to have `Gurobi.jl` installed with a valid license, otherwise replace `Gurobi` and `Gurobi.Optimizer` from `src/LocationTransportation.jl` for an open source alternative, such as `GLPK.jl` :

```julia
using GLPK
solver = optimizer_with_attributes(GPLK.Optimizer)
```

All the necessary dependencies will be managed by running the following cell:

In [1]:
using Pkg
Pkg.activate(@__DIR__)
using LocationTransportation
using DataFrames

  Activating project at `~/Documents/Tech/Academic/M2 ROAD/Courses/3. Optimisation dans l'incertain/Projet/LocationTransportation`


## Question 1: Deterministic model

Let the set of locations be denoted by L and the set of customers by J. The fixed cost of opening a warehouse at location $i \in I$ is denoted by $f_i$ and the cost per unit of capacity is denoted by $a_{i}$, which is limited by a maximum capacity $C_{i}$. The quantity of product demanded by customer j is denoted by $d_j$, and the cost of transporting one unit of product from location i to customer j is denoted by $c_{ij}$. It is assumed that the total available capacity is sufficient to satisfy the total demand to ensure feasibility:

$$\sum_{i \in I} C_{i} \geq \sum_{j \in J} d_{j}$$

We seek to minimize the total cost while satisfying the demand of each customer. To achieve so, define the location decision variable $y_{i} \in \{0,1\}$, which is equal to 1 if a warehouse is opened at location $i \in I$, and 0 otherwise. The capacity decision variable $z_{i} \geq 0$ denotes the warehouse capacity installed at location $i \in I$. The transportation decision variable $x_{ij} \geq 0$ denotes the quantity of product transported from location $i \in I$ to customer $j \in J$. The warehouse location-transportation problem with single product can be formulated in the deterministic form as follows:

$$\min \sum_{i \in I} f_{i} y_{i} + \sum_{i \in I} a_{i} z_{i} + \sum_{i \in I} \sum_{j \in J} c_{ij} x_{ij}$$

$$\text{s.t.} \quad z_{i} \leq C_{i} y_{i}, \quad \forall i \in I \quad \text{(capacity limit)}$$
$$\sum_{j \in J} x_{ij} \leq z_{i}, \quad \forall i \in I \quad \text{(transportation limit)}$$
$$\sum_{i \in I} x_{ij} \geq d_{j}, \quad \forall j \in J \quad \text{(demand satisfaction)}$$
$$y_{i} \in \{0,1\}, \quad \forall i \in I$$
$$z_{i} \geq 0, \quad \forall i \in I$$
$$x_{ij} \geq 0, \quad \forall i \in I, \forall j \in J$$

In this model, the objective function minimizes the total cost, which is the sum of the fixed, capacity, and transportation costs. The capacity limit constraint ensures that the capacity installed does not exceed the maximum capacity if a warehouse is opened at location $i \in I$. The transportation limit constraint guarantees that the quantity of product transported from location $i \in I$ does not exceed the installed capacity. The demand satisfaction constraint provides a lower bound on the quantity of product transported from location $i \in I$ to customer $j \in J$. Finally, the location decision variable $y_{i}$ is set as binary, and the capacity and transportation decision variables $z_{i}$ and $x_{ij}$ are non-negative.

The implementation of this model is located at `src/methods/Base.jl`. In the `solve` function, the `is_opened` variable represent the location decision $y_{i}$; `installed_capacity`, the capacity decision $z_{i}$; and `distribution_amount`, the transportation $x_{ij}$ variable. To solve the problem using the sample instance, call the `execute` function with the desired method:

In [2]:
base_solution = execute(; model=:Base);

┌ Info: Solving instance with Base
└ @ LocationTransportation /Users/sebastiangrandaaltamirano/Documents/Tech/Academic/M2 ROAD/Courses/3. Optimisation dans l'incertain/Projet/LocationTransportation/src/LocationTransportation.jl:44


Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-19
Set parameter TimeLimit to value 3600


The optimal solution reports an objective value of `30536` units of cost:

In [3]:
base_summary = LocationTransportation.summary(base_solution)
@assert base_solution.metrics.objective_value == 30536.0
select(base_summary[3], :objective_value, :execution_time)

Row,objective_value,execution_time
,Float64,Float64
1,30536.0,1.70262


This solution shows that the locations `1` and `3` have been opened, with a capacity of `426.0` and `274`, respectively:

In [4]:
base_summary[1]

Row,location_id,installed_capacity
,String,Float64
1,L-1,426.0
2,L-3,274.0


We can also verify the customer-to-warehouse assignments and the transportation quantity. For this instance, the warehouse `1` delivers `206` and `220` units to customers `1` and `3`, respectively; while warehouse `3` delivers `274` units to customer `2`:

In [5]:
base_summary[2]

Row,warehouse_id,customer_id,distribution_amount
,String,String,Float64
1,L-1,C-1,206.0
2,L-1,C-3,220.0
3,L-3,C-2,274.0


We can verify that this solution satisfies all the demand and it is consistent with the transportation quantities:

In [6]:
@assert sum(base_summary[1][!, :installed_capacity]) == sum(base_summary[2][!, :distribution_amount]) == sum([206, 274, 220])

## Question 2: Robust counterpart

Considering the uncertainty set $D$, in which $\bar{d}_{j}$ is the nominal value of the demand quantity for customer $j \in J$, and $\hat{d}_{j}$ is the maximum deviation from the nominal value, define the decision variables $r_{j}$ as the robust demand quantity for customer $j \in J$ and $\delta_{j}$ as the deviation proportion.
Then, the mathematical formulation to obtain the worst-case demand for each customer is as follows:

$$\max \sum_{j} r_{j}$$

$$r_{j} = \bar{d}_{j} + \hat{d}_{j} \delta_{j}, \quad \forall j \in J$$

$$0 \leq \delta_{j} \leq 1, \quad \forall j \in J$$
    
$$\sum_{j} \delta_{j} \leq \Gamma_{1}$$

$$\delta_{1} + \delta_{2} \leq \Gamma_{2}$$

To introduce this uncertainty set in the model, replace the demand parameter $d_{j}$ with the dual of the demand satisfaction constraint $\lambda_{j}$ multiplied by the robust demand quantity $r_{j}$ :
$$d_{j} = \lambda_{j} r_{j}, \quad \forall j \in J$$

The robust demand quantity $r_{j}$ is obtained by solving the above model.

The implementation for the robust model and for the uncertainty set are located at `src/methods/Robust.jl` and `src/methods/UncertaintySet.jl`, respectively.

In [7]:
robust_solution = execute(; model=:Robust, uncertain_demand=true);

┌ Info: Solving instance with Robust
└ @ LocationTransportation /Users/sebastiangrandaaltamirano/Documents/Tech/Academic/M2 ROAD/Courses/3. Optimisation dans l'incertain/Projet/LocationTransportation/src/LocationTransportation.jl:44


Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-19
Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-19
Set parameter TimeLimit to value 3600
Set parameter TimeLimit to value 3600


The optimal solution reports an objective value of `33496` units of cost:

In [8]:
robust_summary = LocationTransportation.summary(robust_solution)
@assert robust_solution.metrics.objective_value == 33496.0
select(robust_summary[3], :objective_value, :execution_time)

Row,objective_value,execution_time
,Float64,Float64
1,33496.0,0.00181676


As before, the robust solution shows that the locations `1` and `3` have been opened. However, in this solution the location `1` has `498` units of capacity (additional `72` units) and the location `3` remains with the same capacity of `274` :

In [9]:
robust_summary[1]

Row,location_id,installed_capacity
,String,Float64
1,L-1,498.0
2,L-3,274.0


For this solution, the warehouse `1` delivers additional `32` units to customer `1` (`238` vs `206`), an additional of `40` units for customer `3` (`260` vs `220`). The warehouse `3` delivers the same amount to customer `2`.

In [10]:
robust_summary[2]

Row,warehouse_id,customer_id,distribution_amount
,String,String,Float64
1,L-1,C-1,238.0
2,L-1,C-3,260.0
3,L-3,C-2,274.0


To verify that the robust solution is more expensive than the base solution because of the robustness constraint, we can check that the robust solution is at most twice as expensive as the base solution:

In [11]:
@assert base_solution.metrics.objective_value <= robust_solution.metrics.objective_value <= 2 * base_solution.metrics.objective_value

## Question 3: Dynamic constraint generation

Using the same principle as the Benders decomposition method, we obtain the master problem (MP) by determing the location and the capacity of the warehouses. Once this first-stage decisions are made, we solve the subproblem (SP) **for each customer** to determine the transportation quantities from the warehouses to the customers.

In the implementation, the master problem provides candidate solutions and the subproblem either improves the lower bound (optimality cuts) or makes the problem feasible (feasibility cuts). These cuts are added to the master problem using the dual solutions of each subproblem. Let the fixed location and capacity obtained from the master problem be $\bar{y}_{i}$ and $\bar{z}_{i}$, respectively, and considering $r_{j}$ as the robust demand for customer $j$, the subproblem is formulated as follows:

For the **optimality cuts**, we formulate the subproblem as follows:

$$\text{SP-opt}^{j} = \min \sum_{i \in I} c_{ij} x_{ij}$$

$$\text{s.t.} x_{ij} \leq \bar{z}_{i} \quad \forall i \in I \quad \text{(transportation limit)}$$
$$\sum_{i \in I} x_{ij} \geq r_{j} \quad \text{(demand satisfaction)}$$
$$x_{ij} \geq 0 \quad \forall i \in I$$

Given that $\alpha_{i}$ is the dual variable associated with the transportation limit constraint, and $v(\alpha)$ is the optimal objective value of the subproblem, the optimality cut for customer $j$ is formulated as follows (multi-cut approach):

$$v(\alpha) + \sum_{i \in I} \alpha_{i} * z_{i} \leq \theta_{j}$$

On the other side, for the **feasibility cuts** we formulate the auxiliary problem to determine the capacity deficit and surplus of each warehouse. Let $z^{+}_{i}$ and $z^{-}_{i}$ be the artificial variables that represent the capacity surplus and deficit of warehouse $i$, respectively, the auxiliary problem is formulated as:

$$\text{SP-feas}^{j} = \min \sum_{i \in I} z^{+}_{i} + z^{-}_{i}$$

$$\text{s.t.} x_{ij} + z^{-}_{i} - z^{+}_{i} \leq \bar{z}_{i} \quad \forall i \in I \quad \text{(transportation limit)}$$

$$\sum_{i \in I} x_{ij} \geq r_{j} \quad \text{(demand satisfaction)}$$

$$x_{ij} \geq 0 \quad \forall i \in I$$

Given that $\beta_{i}$ is the dual variable associated with the transportation limit constraint, and $v(\beta)$ is the optimal objective value of the subproblem, the feasibility cut is formulated as follows:

$$v(\beta) + \sum_{i \in I} \beta_{i} * z_{i} \leq 0$$

The master problem approximates the subproblem solutions by introducing an additional variable $\theta_{i} \forall i \in I$ that represents the second-stage cost. The master problem is formulated as follows:

$$\min \sum_{i \in I} f_{i} y_{i} + \sum_{i \in I} a_{i} z_{i} + \sum_{i \in I} \theta_{i}$$

$$\text{s.t.} \quad z_{i} \leq C_{i} y_{i}, \quad \forall i \in I \quad \text{(capacity limit)}$$
$$\text{feasibility cuts}$$
$$\text{optimality cuts}$$
$$y_{i} \in \{0, 1\} \quad \forall i \in I$$
$$z_{i} \geq 0 \quad \forall i \in I$$
$$\theta_{i} \geq 0 \quad \forall i \in I$$

For illustrative purposes, the constraint generation method is implemented using an iterative multi-cut approach. The `src/methods/Benders/MasterProblem.jl` file contains the logic related to the master problem formulation and cut generation.
Similarly, the `src/methods/Benders/SubProblem.jl` file contains the logic related to the optimality and feasibility subproblem formulations. The logic is orchestrated in the `src/methods/Benders/ConstraintGeneration.jl` file.

Each sub-problem is solved in parallel to find new cuts for each customer.
For simplicity, the stop criteria of the dynamic constraint generation algorithm is only based on the number of iterations.

In [12]:
cg_solution = execute(; model=:ConstraintGeneration, uncertain_demand=true);

┌ Info: Solving instance with ConstraintGeneration
└ @ LocationTransportation /Users/sebastiangrandaaltamirano/Documents/Tech/Academic/M2 ROAD/Courses/3. Optimisation dans l'incertain/Projet/LocationTransportation/src/LocationTransportation.jl:44
┌ Info: ConstraintGeneration | Iteration 1
└ @ LocationTransportation /Users/sebastiangrandaaltamirano/Documents/Tech/Academic/M2 ROAD/Courses/3. Optimisation dans l'incertain/Projet/LocationTransportation/src/methods/Benders/ConstraintGeneration.jl:17


Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-19
Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-19
Set parameter TimeLimit to value 3600
Set parameter TimeLimit to value 3600


┌ Info: MasterProblem solved and updated | Metrics objective_value: 0.0, execution_time: 0.000299679, expected_recourse: 0
└ @ LocationTransportation /Users/sebastiangrandaaltamirano/Documents/Tech/Academic/M2 ROAD/Courses/3. Optimisation dans l'incertain/Projet/LocationTransportation/src/methods/Benders/MasterProblem.jl:72


Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-19
Set parameter TimeLimit to value 3600
Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-19
Set parameter TimeLimit to value 3600
Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-19
Set parameter TimeLimit to value 3600
Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-19
Set parameter TimeLimit to value 3600
Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-19
Set parameter TimeLimit to value 3600
Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-19
Set parameter TimeLimit to value 3600


┌ Warning: Model not optimal
└ @ LocationTransportation /Users/sebastiangrandaaltamirano/Documents/Tech/Academic/M2 ROAD/Courses/3. Optimisation dans l'incertain/Projet/LocationTransportation/src/services/helpers.jl:8
┌ Warning: Model not optimal
└ @ LocationTransportation /Users/sebastiangrandaaltamirano/Documents/Tech/Academic/M2 ROAD/Courses/3. Optimisation dans l'incertain/Projet/LocationTransportation/src/services/helpers.jl:8
┌ Warning: Model not optimal
└ @ LocationTransportation /Users/sebastiangrandaaltamirano/Documents/Tech/Academic/M2 ROAD/Courses/3. Optimisation dans l'incertain/Projet/LocationTransportation/src/services/helpers.jl:8


...

Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-19
Set parameter TimeLimit to value 3600
Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-19
Set parameter TimeLimit to value 3600
Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-19
Set parameter TimeLimit to value 3600
Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-19
Set parameter TimeLimit to value 3600
Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-19
Set parameter TimeLimit to value 3600
Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-19
Set parameter TimeLimit to value 3600


┌ Info: Location 1 | Optimality cut added: -θ[1] ≤ -4760
└ @ LocationTransportation /Users/sebastiangrandaaltamirano/Documents/Tech/Academic/M2 ROAD/Courses/3. Optimisation dans l'incertain/Projet/LocationTransportation/src/methods/Benders/MasterProblem.jl:100
┌ Info: Location 2 | Optimality cut added: -2 installed_capacity[2] - θ[2] ≤ -6850
└ @ LocationTransportation /Users/sebastiangrandaaltamirano/Documents/Tech/Academic/M2 ROAD/Courses/3. Optimisation dans l'incertain/Projet/LocationTransportation/src/methods/Benders/MasterProblem.jl:100
┌ Info: Location 3 | Optimality cut added: -3 installed_capacity[1] - θ[3] ≤ -7020
└ @ LocationTransportation /Users/sebastiangrandaaltamirano/Documents/Tech/Academic/M2 ROAD/Courses/3. Optimisation dans l'incertain/Projet/LocationTransportation/src/methods/Benders/MasterProblem.jl:100
┌ Info: ConstraintGeneration | Iteration 5
└ @ LocationTransportation /Users/sebastiangrandaaltamirano/Documents/Tech/Academic/M2 ROAD/Courses/3. Optimisation dans l

Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-19
Set parameter TimeLimit to value 3600
Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-19
Set parameter TimeLimit to value 3600
Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-19
Set parameter TimeLimit to value 3600
Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-19
Set parameter TimeLimit to value 3600
Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-19
Set parameter TimeLimit to value 3600
Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-19
Set parameter TimeLimit to value 3600


┌ Info: Location 1 | Optimality cut added: -θ[1] ≤ -4760
└ @ LocationTransportation /Users/sebastiangrandaaltamirano/Documents/Tech/Academic/M2 ROAD/Courses/3. Optimisation dans l'incertain/Projet/LocationTransportation/src/methods/Benders/MasterProblem.jl:100
┌ Info: Location 2 | Optimality cut added: -2 installed_capacity[2] - θ[2] ≤ -6850
└ @ LocationTransportation /Users/sebastiangrandaaltamirano/Documents/Tech/Academic/M2 ROAD/Courses/3. Optimisation dans l'incertain/Projet/LocationTransportation/src/methods/Benders/MasterProblem.jl:100
┌ Info: Location 3 | Optimality cut added: -3 installed_capacity[1] - θ[3] ≤ -7020
└ @ LocationTransportation /Users/sebastiangrandaaltamirano/Documents/Tech/Academic/M2 ROAD/Courses/3. Optimisation dans l'incertain/Projet/LocationTransportation/src/methods/Benders/MasterProblem.jl:100
┌ Info: MasterProblem solved and updated | Metrics objective_value: 24436.0, execution_time: 0.001933317, expected_recourse: 0
└ @ LocationTransportation /Users/seba

┌ Info: ConstraintGeneration | History
│   master.history = LocationTransportation.Metrics[LocationTransportation.Metrics(0.0, 0.000299679, 0), LocationTransportation.Metrics(5332.0, 0.001126438, 0), LocationTransportation.Metrics(23584.0, 0.004421991, 0), LocationTransportation.Metrics(24436.0, 0.001461874, 0), LocationTransportation.Metrics(24436.0, 0.001520221, 0), LocationTransportation.Metrics(24436.0, 0.001933317, 0)]
└ @ LocationTransportation /Users/sebastiangrandaaltamirano/Documents/Tech/Academic/M2 ROAD/Courses/3. Optimisation dans l'incertain/Projet/LocationTransportation/src/methods/Benders/ConstraintGeneration.jl:27


The optimal solution reports an objective value of `30536` units of cost:

In [13]:
cg_summary = LocationTransportation.summary(cg_solution)
@assert cg_solution.metrics.objective_value == 24436.0
select(cg_summary[3], :objective_value, :execution_time)

Row,objective_value,execution_time
,Float64,Float64
1,24436.0,26.674


This solution shows that the locations `1` and `3` have been opened, with a capacity of `426.0` and `274`, respectively:

In [14]:
cg_summary[1]

Row,location_id,installed_capacity
,String,Float64
1,L-3,274.0


We can also verify the customer-to-warehouse assignments and the transportation quantity. For this instance, the warehouse `1` delivers `206` and `220` units to customers `1` and `3`, respectively; while warehouse `2` delivers `274` units to customer `2`:

In [15]:
cg_summary[2]

Row,warehouse_id,customer_id,distribution_amount
,String,String,Float64
1,L-3,C-1,4760.0
2,L-3,C-2,4760.0
3,L-3,C-3,4760.0


In [16]:
@assert sum(cg_summary[1][!, :installed_capacity]) == sum(cg_summary[2][!, :distribution_amount]) == sum([206, 274, 220])

# TODO verificar... aparentemente solo abre 1.

AssertionError: AssertionError: sum((cg_summary[1])[!, :installed_capacity]) == sum((cg_summary[2])[!, :distribution_amount]) == sum([206, 274, 220])

## Question 4: Affine decision rule

The affine decision rule states that the second-stage decisions, which are the transportation quantities, are affine functions of the uncertain demand.
In particular, this decision rule approximates the $x_{ij}$ variables as an affine function of the uncertain demand $d_{j}(\xi)$ of customer $j \in J$.
We introduce two additional variables, $x_{ij}^{0}$ and $x_{ij}^{k}$, which represent the nominal value of the transportation quantity and the impact of the uncertain demand $d_{k}(\xi)$ of customer $k \in J$ from warehouse $i \in I$ to customer $j \in J$, respectively.
Then, we can express the transportation quantity from warehouse $i \in I$ to customer $j \in J$ as follows:

$$x_{ij}(\xi) = x_{ij}^{0} + \sum_{k \in J} x_{ij}^{k} d_{k}(\xi)$$

To formulate the rule under the functional form, we modify the demand satisfaction constraint, resulting in the following formulation:

$$\min \sum_{i \in I} f_{i} y_{i} + \sum_{i \in I} a_{i} z_{i} + \sum_{i \in I} \sum_{j \in J} c_{ij} x_{ij}$$

$$\text{s.t.} \quad x_{ij}(\xi) \geq x_{ij}^{0} + \sum_{k \in J} x_{ij}^{k} d_{k}(\xi) \quad \forall i \in I, j \in J \quad \text{(affine rule)}$$
$$\text{s.t.} \quad z_{i} \leq C_{i} y_{i}, \quad \forall i \in I \quad \text{(capacity limit)}$$
$$\sum_{j \in J} x_{ij}(\xi) \leq z_{i}, \quad \forall i \in I \quad \text{(transportation limit)}$$
$$\sum_{i \in I} x_{ij}(\xi) \geq d_{j}(\xi), \quad \forall j \in J \quad \text{(affine demand satisfaction)}$$
$$x_{ij}(\xi) \geq 0, \quad \forall i \in I, j \in J$$
$$z_{i} \geq 0, \quad \forall i \in I$$
$$y_{i} \in \{0,1\}, \quad \forall i \in I$$

The deterministic formulation of this problem is obtained by replacing the uncertain demand $d_{j}(\xi)$ by the robust demand $r_{j}$ obtained from the uncertainty set.

$$\min \sum_{i \in I} f_{i} y_{i} + \sum_{i \in I} a_{i} z_{i} + \sum_{i \in I} \sum_{j \in J} c_{ij} x_{ij}$$

$$\text{s.t.} \quad x_{ij} \geq x_{ij}^{0} + \sum_{k \in J} x_{ij}^{k} r_{k} \quad \forall i \in I, j \in J \quad \text{(affine rule)}$$
$$\text{s.t.} \quad z_{i} \leq C_{i} y_{i}, \quad \forall i \in I \quad \text{(capacity limit)}$$
$$\sum_{j \in J} x_{ij} \leq z_{i}, \quad \forall i \in I \quad \text{(transportation limit)}$$
$$\sum_{i \in I} x_{ij} \geq r_{j}, \quad \forall j \in J \quad \text{(affine demand satisfaction)}$$
$$x_{ij} \geq 0, \quad \forall i \in I, j \in J$$
$$z_{i} \geq 0, \quad \forall i \in I$$
$$y_{i} \in \{0,1\}, \quad \forall i \in I$$

The implementation of the affine decision rule can be found at `src/methods/AffineDecision.jl`.

In [17]:
affine_solution = execute(; model=:AffineDecision, uncertain_demand=true);

┌ Info: Solving instance with AffineDecision
└ @ LocationTransportation /Users/sebastiangrandaaltamirano/Documents/Tech/Academic/M2 ROAD/Courses/3. Optimisation dans l'incertain/Projet/LocationTransportation/src/LocationTransportation.jl:44


Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-19
Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-19
Set parameter TimeLimit to value 3600
Set parameter TimeLimit to value 3600


The optimal solution reports an objective value of `33496` units of cost, which is less than the `Robust` method. This is expected because the **affine decision rule provides less conservative solutions as they can be adjusted according to the realized proportion of data at a given stage**.

In [18]:
affine_summary = LocationTransportation.summary(affine_solution)
@assert affine_solution.metrics.objective_value == 33496.0
select(affine_summary[3], :objective_value, :execution_time)

Row,objective_value,execution_time
,Float64,Float64
1,33496.0,0.00192728


This solution shows that the locations `1` and `3` have been opened, with a capacity of `498.0` and `274`, respectively:

In [19]:
affine_summary[1]

Row,location_id,installed_capacity
,String,Float64
1,L-1,498.0
2,L-3,274.0


The warehouse `1` delivers `238` and `260` units to customers `1` and `3`, respectively; while warehouse `3` delivers `274` units to customer `2`:

In [20]:
affine_summary[2]

Row,warehouse_id,customer_id,distribution_amount
,String,String,Float64
1,L-1,C-1,238.0
2,L-1,C-3,260.0
3,L-3,C-2,274.0


Finally, verify that the solution satisfies all the demand and it is consistent with the transportation quantities:

In [21]:
@assert sum(affine_summary[1][!, :installed_capacity]) == sum(affine_summary[2][!, :distribution_amount]) >= sum([206, 274, 220])